# Bitinfo for ERA5-Land data
Here we test each dimension for bitinfo. 

In [ ]:
import xarray as xr
import xbitinfo as xb
import hvplot.xarray
import fsspec 

Copy netcdf file obtained using the cdsapi service from cloud to local disk:

In [ ]:
s3file = 's3://rsignellbucket1/testing/conus_2019-12-01.nc'
local_file = 'conus_2019-12-01.nc'
local_compressed_file = local_file.replace('.nc','_br.nc')

In [ ]:
fs = fsspec.filesystem('s3', anon=True, 
                       client_kwargs={'endpoint_url': 'https://mghp.osn.xsede.org'})
fs.download(s3file, local_file)

Explore local file

In [ ]:
ds = xr.open_dataset(local_file)

In [ ]:
ds

In [ ]:
ds.hvplot(x='longitude', y='latitude', geo=True, rasterize=True)

Subset a region to get all land (bitinfo gives misleading answers if masked areas are included)

In [ ]:
ds_subset = ds.sel(longitude=slice(-120,-93),latitude=slice(50,38))
ds_subset.hvplot(x='longitude', y='latitude', geo=True, rasterize=True)

In [ ]:
bitinfo = xb.get_bitinformation(ds_subset, dim="latitude")  
keepbits = xb.get_keepbits(bitinfo, inflevel=0.99)  # get number of mantissa bits to keep for 99% real information
keepbits

In [ ]:
bitinfo = xb.get_bitinformation(ds_subset, dim="longitude")  
keepbits = xb.get_keepbits(bitinfo, inflevel=0.99)  # get number of mantissa bits to keep for 99% real information
keepbits

In [ ]:
bitinfo = xb.get_bitinformation(ds_subset, dim="time")  # calling bitinformation.jl.bitinformation
keepbits = xb.get_keepbits(bitinfo, inflevel=0.99)  # get number of mantissa bits to keep for 99% real information
keepbits

In [ ]:
ds_bitrounded = xb.xr_bitround(ds, keepbits)  # bitrounding keeping only keepbits mantissa bits

In [ ]:
%%time
ds_bitrounded.to_compressed_netcdf(local_compressed_file)  # save to netcdf with compression

In [ ]:
! du -h *.nc

In [ ]:
keepbits.to_netcdf('keepbits.nc')